In [1]:
from __future__ import division
from __future__ import print_function
from utils import *
import tensorflow as tf
import numpy as np
import os
import random
import scipy.sparse as sp
import networkx as nx
# Set random seed
seed = 123
np.random.seed(seed)
tf.random.set_seed(seed)


####################################################################
#################INIZIALISATION ###############

conv=16

####################################################################

adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data('cora')
adjfeatures_arr=sparse_to_tuple(features)
support = preprocess_adj(adj)

features=features.toarray()

###############################################################################################################################################################
###############################################################################################################################################################


In [2]:
#adj as sparse tensor
adj_tf=tf.sparse.SparseTensor(support[0], support[1], support[2])
adj_tf=tf.dtypes.cast(adj_tf,tf.float32) 
#adj_tf=tf.sparse.to_dense(tf.sparse.reorder(adj_tf))
#features as sparse tensor
features_tf=tf.convert_to_tensor(features, dtype=tf.float32)
# cast to tensorflow all arr
y_train_tf=tf.convert_to_tensor(y_train, dtype=tf.float32)
train_mask_tf=tf.convert_to_tensor(train_mask)


In [3]:
train_mask_tf

<tf.Tensor: shape=(2708,), dtype=bool, numpy=array([ True,  True,  True, ..., False, False, False])>

In [4]:
import GCNLayer
import sonnet as snt
learning_rate=0.01

model=GCNLayer.GCN(convolution_kernel_1=conv,convolution_kernel_2=np.shape(y_train)[1])
optimizer = tf.keras.optimizers.Adam(learning_rate)



from metrics import masked_softmax_cross_entropy

def ComputeLoss(labels, preds,mask):
    _loss=masked_softmax_cross_entropy(preds, labels, mask)
    _masked=_loss
    _loss+= tf.nn.l2_loss(model.trainable_variables[-1])*5e-4
    return _loss,_masked


def update_step(inputs_tr,adj, targets_tr,masks):
    with tf.GradientTape() as tape:
        output_ops_tr = model(inputs_tr,adj)
        # Loss.
        loss_tr,masked_tr = ComputeLoss(targets_tr,output_ops_tr,masks)
        

    gradients = tape.gradient(loss_tr, model.trainable_variables)
    #grads = tf.distribute.get_replica_context().all_reduce('sum', gradients)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    #optimizer.apply(gradients, model.trainable_variables)
    return loss_tr,masked_tr


In [5]:
input_signature = [tf.TensorSpec.from_tensor(features_tf),tf.SparseTensorSpec(shape=adj_tf.get_shape(),dtype=tf.dtypes.float32),tf.TensorSpec.from_tensor(y_train_tf),tf.TensorSpec.from_tensor(train_mask_tf)]
compiled_update_step = tf.function(update_step, input_signature=input_signature)

In [6]:


# Train model
for epoch in range(200):
    loss_tr,masked_tr = compiled_update_step(features_tf, adj_tf, y_train_tf,train_mask_tf)
    
    if(epoch%1==0):
        print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(loss_tr.numpy()))



Epoch: 0001 train_loss= 1.96274
Epoch: 0002 train_loss= 1.84639
Epoch: 0003 train_loss= 1.73923
Epoch: 0004 train_loss= 1.58959
Epoch: 0005 train_loss= 1.44386
Epoch: 0006 train_loss= 1.35270
Epoch: 0007 train_loss= 1.19640
Epoch: 0008 train_loss= 1.07133
Epoch: 0009 train_loss= 0.97654
Epoch: 0010 train_loss= 0.92415
Epoch: 0011 train_loss= 0.84775
Epoch: 0012 train_loss= 0.71689
Epoch: 0013 train_loss= 0.63866
Epoch: 0014 train_loss= 0.62385
Epoch: 0015 train_loss= 0.50535
Epoch: 0016 train_loss= 0.49537
Epoch: 0017 train_loss= 0.48774
Epoch: 0018 train_loss= 0.44343
Epoch: 0019 train_loss= 0.41045
Epoch: 0020 train_loss= 0.36089
Epoch: 0021 train_loss= 0.30845
Epoch: 0022 train_loss= 0.29369
Epoch: 0023 train_loss= 0.32588
Epoch: 0024 train_loss= 0.27005
Epoch: 0025 train_loss= 0.27608
Epoch: 0026 train_loss= 0.21855
Epoch: 0027 train_loss= 0.19943
Epoch: 0028 train_loss= 0.19729
Epoch: 0029 train_loss= 0.16722
Epoch: 0030 train_loss= 0.17609
Epoch: 0031 train_loss= 0.16461
Epoch: 0

In [16]:
loss_tr.numpy()

0.06629568